# 04 - Live Scoring & Dashboard

Scoring en vivo de mercados activos de Polymarket.

Este notebook cubre:
- Carga del modelo entrenado
- Scoring de mercados activos en tiempo real
- Generación de señales de compra
- Dashboard visual de oportunidades
- Backtesting con datos históricos

In [ ]:
import sys
sys.path.insert(0, '..')

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from pathlib import Path

from src.data.client import PolymarketDataClient
from src.features.pipeline import FeaturePipeline
from src.model.architecture import MarketValueNet
from src.scoring.scorer import score_active_markets
from src.scoring.signals import generate_signals, format_signals_report

sns.set_theme(style='whitegrid')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

## 1. Cargar modelo y pipeline

In [ ]:
# Cargar pipeline de features
pipeline = FeaturePipeline.load('../data/processed/pipeline', use_dummy_text=True)

# Cargar modelo
model = MarketValueNet(
    num_numerical_features=pipeline.num_numerical_features,
    num_categories=pipeline.num_categories,
    text_embed_dim=pipeline.text_embed_dim,
)
model.load_state_dict(torch.load('../data/models/best_market_model.pt', weights_only=True))
model.eval()

print('Modelo y pipeline cargados.')

## 2. Scoring de mercados activos

In [ ]:
# Cliente de API
client = PolymarketDataClient()

# Scoring
scored_df = score_active_markets(
    model=model,
    client=client,
    feature_pipeline=pipeline,
    device=device,
    top_k=30,
    max_markets=500,
)

print(f'Mercados puntuados: {len(scored_df)}')
scored_df.head(10)

## 3. Generar señales

In [ ]:
# Generar señales de trading
signals_df = generate_signals(
    scored_df,
    buy_threshold=0.6,
    strong_buy_threshold=0.75,
    min_liquidity=1000,
    min_volume_24h=100,
    max_spread=0.10,
)

# Reporte
report = format_signals_report(signals_df)
print(report)

## 4. Dashboard Visual

In [ ]:
# Top oportunidades de compra
fig, ax = plt.subplots(figsize=(14, 8))
top_markets = scored_df.head(15).copy()

if not top_markets.empty:
    colors = plt.cm.RdYlGn(top_markets['model_score'].values)
    
    y_labels = top_markets['question'].str[:55].values
    bars = ax.barh(range(len(top_markets)), top_markets['model_score'], color=colors)
    ax.set_yticks(range(len(top_markets)))
    ax.set_yticklabels(y_labels, fontsize=9)
    ax.set_xlabel('Model Score (probabilidad de ser buena compra)')
    ax.set_title('Top 15 Oportunidades de Compra en Polymarket')
    ax.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5, label='Threshold')
    
    # Añadir precio actual como referencia
    for i, (_, row) in enumerate(top_markets.iterrows()):
        ax.annotate(
            f'${row["price_yes"]:.2f}',
            xy=(row['model_score'] + 0.01, i),
            fontsize=8, color='gray', va='center'
        )
    
    ax.legend()
    ax.invert_yaxis()

plt.tight_layout()
plt.savefig('../figures/04_top_opportunities.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Score vs Precio
fig, ax = plt.subplots(figsize=(10, 7))

if not scored_df.empty:
    scatter = ax.scatter(
        scored_df['price_yes'],
        scored_df['model_score'],
        c=scored_df['expected_alpha'],
        cmap='RdYlGn',
        s=scored_df['volume_24h'].clip(upper=scored_df['volume_24h'].quantile(0.95)) / 100 + 20,
        alpha=0.7,
        edgecolors='gray', linewidths=0.5
    )
    
    ax.plot([0, 1], [0, 1], 'k--', alpha=0.3, label='Score = Price (sin alpha)')
    ax.set_xlabel('Precio Yes (probabilidad de mercado)')
    ax.set_ylabel('Model Score (probabilidad estimada)')
    ax.set_title('Score del Modelo vs Precio de Mercado\n(color = alpha esperado, tamaño = volumen)')
    plt.colorbar(scatter, label='Expected Alpha')
    ax.legend()

plt.tight_layout()
plt.savefig('../figures/04_score_vs_price.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Señales por tipo
if not signals_df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Conteo de señales
    signal_counts = signals_df['signal'].value_counts()
    colors_map = {'STRONG BUY': 'darkgreen', 'BUY': 'limegreen', 'HOLD': 'gray'}
    bar_colors = [colors_map.get(s, 'gray') for s in signal_counts.index]
    signal_counts.plot.bar(ax=axes[0], color=bar_colors)
    axes[0].set_title('Distribución de Señales')
    axes[0].set_ylabel('Cantidad')
    plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=0)
    
    # Alpha esperado por señal
    signals_df.boxplot(column='expected_alpha', by='signal', ax=axes[1])
    axes[1].set_title('Alpha Esperado por Tipo de Señal')
    axes[1].set_ylabel('Expected Alpha')
    plt.suptitle('')
    
    plt.tight_layout()
    plt.show()

## 5. Backtesting

In [ ]:
from src.model.evaluate import backtest

# Cargar mercados resueltos para backtesting
resolved_path = Path('../data/raw/resolved_markets.json')
if resolved_path.exists():
    with open(resolved_path) as f:
        resolved_markets = json.load(f)
    
    # Parsear mercados
    from src.data.client import PolymarketDataClient
    resolved_parsed = [PolymarketDataClient.parse_market(m) for m in resolved_markets[:200]]
    
    # Backtesting
    trades_df, final_capital = backtest(
        model=model,
        historical_markets=resolved_parsed,
        feature_pipeline=pipeline,
        initial_capital=1000.0,
        position_size=0.05,
        threshold=0.6,
        device=device,
    )
    
    print(f'=== Resultados de Backtesting ===')
    print(f'Capital inicial:  $1,000.00')
    print(f'Capital final:    ${final_capital:,.2f}')
    print(f'Retorno:          {(final_capital/1000 - 1)*100:.1f}%')
    print(f'Total trades:     {len(trades_df)}')
    if not trades_df.empty:
        print(f'Win rate:         {(trades_df["pnl"] > 0).mean():.1%}')
        print(f'Avg PnL:          ${trades_df["pnl"].mean():.2f}')
        print(f'Best trade:       ${trades_df["pnl"].max():.2f}')
        print(f'Worst trade:      ${trades_df["pnl"].min():.2f}')
else:
    print('No se encontraron mercados resueltos. Ejecuta primero la descarga de datos.')

In [ ]:
# Curva de capital
if resolved_path.exists() and not trades_df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Equity curve
    axes[0].plot(trades_df['capital_after'].values, linewidth=2, color='steelblue')
    axes[0].axhline(y=1000, color='gray', linestyle='--', alpha=0.5, label='Capital inicial')
    axes[0].set_title('Curva de Capital (Backtesting)')
    axes[0].set_xlabel('Trade #')
    axes[0].set_ylabel('Capital ($)')
    axes[0].legend()
    
    # PnL distribution
    axes[1].hist(trades_df['pnl'], bins=30, color='steelblue', alpha=0.7, edgecolor='black')
    axes[1].axvline(x=0, color='red', linestyle='--')
    axes[1].set_title('Distribución de PnL por Trade')
    axes[1].set_xlabel('PnL ($)')
    axes[1].set_ylabel('Frecuencia')
    
    plt.tight_layout()
    plt.savefig('../figures/04_backtest_results.png', dpi=150, bbox_inches='tight')
    plt.show()